In [1]:
import pandas as pd 
import numpy as np 
import sqlalchemy
from sqlalchemy import create_engine
import pyodbc
import sqlite3
from datetime import datetime

In [2]:
engine = create_engine('sqlite:///chinook.db') 
engine

Engine(sqlite:///chinook.db)

In [ ]:
F_I='2009-01-01'
F_F='2030-12-31'
df_Fecha = pd.DataFrame({'fecha': pd.date_range(F_I, F_F)})
df_Fecha['año'] = df_Fecha.fecha.dt.year
df_Fecha['mes'] = df_Fecha.fecha.dt.month
df_Fecha['dia'] = df_Fecha.fecha.dt.day
df_Fecha['nombre_dia'] = df_Fecha.fecha.dt.day_name()
df_Fecha['dia_semana'] = df_Fecha.fecha.dt.dayofweek
df_Fecha['semana'] = df_Fecha.fecha.dt.weekofyear
df_Fecha['trimestre'] = df_Fecha.fecha.dt.quarter
df_Fecha = df_Fecha[['fecha', 'año', 'mes', 'dia', 'nombre_dia', 'dia_semana', 'semana', 'trimestre']] 


In [5]:
df_Track=pd.read_sql_query("""SELECT  t.Name, mt.Name AS MediaType, g.Name AS Genre, t.Composer, t.Milliseconds, t.Bytes, t.UnitPrice 
       FROM tracks t
       INNER JOIN
       [genres] g ON t.GenreId = g.GenreId
       INNER JOIN
       media_types mt ON t.MediaTypeId = mt.MediaTypeId""" , con=engine.connect())

df_Artists=pd.read_sql_query("""SELECT Name From artists ; """ , con=engine.connect())
df_Customers=pd.read_sql_query("""SELECT  FirstName, LastName, Phone, Fax, Email from customers; """ , con=engine.connect())
df_InvoiceItem=pd.read_sql_query("""SELECT UnitPrice, Quantity from invoice_items ; """ , con=engine.connect())
df_Album=pd.read_sql_query("""SELECT Title From albums ; """ , con=engine.connect())
df_Location=pd.read_sql_query("""SELECT  BillingAddress AS Address, BillingCity AS City, BillingState AS State, BillingCountry AS Country, BillingPostalCode  AS PostalCode from invoices; """ , con=engine.connect())
df_Employee=pd.read_sql_query("""SELECT FirstName, LastName, Title, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email From employees ; """ , con=engine.connect())

df_FactSales = pd.read_sql_query("""SELECT invoice_items.InvoiceLineId as InvoicesID, customers.CustomerId AS CustomerID, employees.EmployeeId AS EmployeeID, dim_time.timeID AS TimeID,
invoices.InvoiceId as LocationID, tracks.TrackId AS TrackId, artists.ArtistId AS ArtistId, albums.AlbumId AS AlbumId, invoices.Total AS Total
FROM invoices INNER JOIN customers
ON invoices.CustomerId= customers.CustomerId
INNER JOIN employees
ON employees.EmployeeId= customers.SupportRepId
INNER JOIN invoice_items
ON invoice_items.InvoiceId= invoices.InvoiceId
INNER JOIN tracks
ON tracks.TrackId=  invoice_items.TrackId
INNER JOIN albums
ON tracks.AlbumId= albums.AlbumId
INNER JOIN artists 
ON albums.ArtistId= artists.ArtistId 
INNER JOIN dim_time
ON invoices.InvoiceDate= substr(dim_time.fecha,0,20);""", con=engine.connect())



In [ ]:
df_Track.fillna(method="ffill", inplace = True) 
df_Customers.fillna(method="ffill", inplace = True) 
df_InvoiceItem.fillna(method="ffill", inplace = True) 
df_Location.fillna(method="ffill", inplace = True) 
df_Employee.fillna(method="ffill", inplace = True) 

In [ ]:
df_Fecha.to_sql(name='Dim_Time', con=engine.connect(), if_exists='append',index=False)

In [ ]:
coneSQL = sqlalchemy.create_engine('mssql+pyodbc://RONYY\SQL2021/DW_Sales_Music?driver=SQL+Server+Native+Client+11.0')
df_Track.to_sql(name='Dim_Track', con=coneSQL, if_exists='append',index=False)
df_Artists.to_sql(name='Dim_Artist', con=coneSQL, if_exists='append',index=False)
df_Customers.to_sql(name='Dim_Customer', con=coneSQL, if_exists='append',index=False)
df_Album.to_sql(name='Dim_Album', con=coneSQL, if_exists='append',index=False)
df_Employee.to_sql(name='Dim_Employee', con=coneSQL, if_exists='append',index=False)
df_Location.to_sql(name='Dim_Location', con=conec, if_exists='append',index=False)
df_Fecha.to_sql(name='Dim_Time', con=coneSQL, if_exists='append',index=False)
df_InvoiceItem.to_sql(name='Dim_Invoice_Item', con=coneSQL, if_exists='append',index=False)
df_FactSales.to_sql(name='Fact_Sales', con=coneSQL, if_exists='append',index=False)